# blah blah

In [1]:

import numpy as np
import pandas as pd

#######################
## Datenaufbereitung ##
#######################


# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path
                                
file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows  = None

## Libraries & Settings ##
from pa_lib.file import load_bin
from pa_lib.util import cap_words
from pa_lib.log import time_log, info

import datetime as dt
from dateutil.relativedelta import relativedelta

from pa_lib.data import (
    clean_up_categoricals,
    unfactorize,
)

from pa_lib.data import desc_col

In [24]:
ek_info = load_bin("vkprog\\ek_info.feather")

2019-10-29 12:11:09 [INFO] Started loading binary file
2019-10-29 12:11:09 [INFO] Reading from file C:\Users\stc\data\vkprog\ek_info.feather
2019-10-29 12:11:09 [INFO] Finished loading binary file in 0.04s (0.05s CPU)


In [25]:
desc_col(ek_info)

,DTYPE,NULLS,UNIQUE
index,int64,0/32237,32237
Endkunde_NR,int64,0/32237,32237
Endkunde,object,0/32237,30291
EK_Aktiv,int64,0/32237,2
EK_Kam_Betreut,int64,0/32237,2
EK_Land,object,49/32188,45
PLZ,int64,0/32237,2875
GEMEINDE,object,49/32188,3006
Agentur,object,20999/11238,3247
EK_BG,object,1829/30408,83


# CRM STUFF

In [69]:
crm_data = load_bin("vkprog\\crm_data.feather").astype({"ENDKUNDE_NR": "int64"})

2019-10-28 14:34:23 [INFO] Started loading binary file
2019-10-28 14:34:23 [INFO] Reading from file C:\Users\stc\data\vkprog\crm_data.feather
2019-10-28 14:34:24 [INFO] Finished loading binary file in 0.21s (0.25s CPU)


In [71]:
display(crm_data.sample(5, random_state=42))
desc_col(crm_data)

,BETREFF,KANAL,STARTTERMIN,QUELLE,VERANTWORTLICH,KUERZEL,ENDKUNDE_NR,VB_FILTER_VON,VB_FILTER_BIS,VB_FILTER_GRUND
490351,Traffic SR-Actions 2017 - Changement d'adresse,E-Mail,2017-12-22,Marketing,Cédric Genoud (TRAFFIC ROMANDIE),CGE,486920,NaT,NaT,NaN
937554,Telemarketing,Besuch,2015-06-23,Verkauf,Massimo Bonotto (REGION OST CHUR),BON,604538,NaT,NaT,NaN
27740,2013_Weihnachtskarten - Mitte_2013,Brief,2013-11-22,Marketing,Maja Loosli (REGION VERKAUF MITTE),MAL,103253,NaT,NaT,NaN
799222,Ouest - Aktionen 2019 - Promotion d'été 2019 -...,E-Mail,2019-05-23,Marketing,Lisa Conte (REGION WEST GENF),LIC,570321,NaT,NaT,NaN
915179,Anfrage Buswerbung,E-Mail,2015-01-13,Verkauf,Domenico Torre (APG WINTERTHUR),DTO,598472,NaT,NaT,NaN


,DTYPE,NULLS,UNIQUE
BETREFF,category,26/1056374,336940
KANAL,category,0/1056400,12
STARTTERMIN,datetime64[ns],0/1056400,3817
QUELLE,category,0/1056400,3
VERANTWORTLICH,category,0/1056400,337
KUERZEL,category,1/1056399,340
ENDKUNDE_NR,int64,0/1056400,51555
VB_FILTER_VON,datetime64[ns],1054692/1708,13
VB_FILTER_BIS,datetime64[ns],1055145/1255,37
VB_FILTER_GRUND,category,1054709/1691,60


I need:
- ``Letzte_VBs``
- ``KZ_letzter_Ktkt`` (Letzter VB-Kontakt, Kürzel)
- ``Kanal`` (Kontaktweg: Email, Telefon, Brieftaube...)
- ``Betreff`` (Inhalt des letzten Kontakts)

In [6]:
crm_data.sort_values("STARTTERMIN", ascending=False).head(5)

,BETREFF,KANAL,STARTTERMIN,QUELLE,VERANTWORTLICH,KUERZEL,ENDKUNDE_NR,VB_FILTER_VON,VB_FILTER_BIS,VB_FILTER_GRUND
624924,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2024-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
69410,A contacter,Besuch,2023-10-21,Verkauf,Michel Zbinden (REGION WEST GENF),MZB,109933,NaT,NaT,NaN
624923,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2023-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
624922,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2022-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
624921,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2021-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN


In [9]:
def collect(s, sep=","):
    return sep.join(map(str, s[s.notna()].unique()))

In [10]:
def last_notna(s):
        try:
            return s.loc[s.notnull()].iat[-1]
        except IndexError:
            return np.NaN

In [93]:
def crm_info():
    today = pd.Timestamp.now()

    ## Information about last contact: Kanal, VB, Betreff
    row_select = (crm_data.loc[:,"STARTTERMIN"] <= today)
    container_crm = (crm_data.loc[row_select,:]
                             .sort_values(["ENDKUNDE_NR","STARTTERMIN"])
                             .groupby("ENDKUNDE_NR", as_index=False)
                             .agg(
                                 {"KUERZEL": last_notna,
                                  "KANAL": last_notna,
                                  "BETREFF": last_notna,
                                  #"STARTTERMIN": last_notna,
                                 }
                             )
                             .rename(columns={
                                 "KUERZEL": "Letzter_Kontakt",
                                 "KANAL": "Kanal",
                                 "BETREFF": "Betreff",
                                 "ENDKUNDE_NR": "Endkunde_NR",
                             })
                    )

    # Letzte CRM-Kontakte (alle der letzten zwei Kalendarjahre)
    row_select = (crm_data.loc[:,"STARTTERMIN"]
                          .apply(lambda x: 2 >= today.isocalendar()[0] - x.isocalendar()[0])
                  & (crm_data.loc[:,"STARTTERMIN"] <= today)
                 )

    crm_letzte_vbs = (crm_data.loc[row_select,:]
                              .groupby("ENDKUNDE_NR", as_index=False)
                              .agg(
                                  {"KUERZEL": collect,
                                  }
                              )
                              .rename(columns={
                                  "KUERZEL": "Letzte_CRM_Ktkts",
                                  "ENDKUNDE_NR": "Endkunde_NR",})
                     )
    # Combine
    container_crm = pd.merge(container_crm, crm_letzte_vbs, on="Endkunde_NR",how="left")

    return container_crm

In [94]:
ek_crm_info = crm_info()

# BD stuff

In [2]:
bookings_raw = load_bin("vkprog\\bd_data.feather").rename(
            mapper=lambda name: cap_words(name, sep="_"), axis="columns"
        )
booking_raw = bookings_raw.loc[(bookings_raw.Netto > 0)].pipe(clean_up_categoricals)

2019-10-29 11:54:25 [INFO] Started loading binary file
2019-10-29 11:54:25 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-29 11:54:25 [INFO] Finished loading binary file in 0.18s (0.88s CPU)


In [3]:
desc_col(booking_raw)

,DTYPE,NULLS,UNIQUE
Endkunde_NR,int64,0/680311,32237
Endkunde,category,0/680311,30291
EK_Abc,category,1155/679156,8
EK_Boni,category,76/680235,3
EK_Plz,category,165/680146,2968
EK_Ort,category,165/680146,3019
EK_Land,category,165/680146,45
EK_HB_Apg_Kurzz,category,28683/651628,91
EK_Kam_Betreut,int64,0/680311,2
EK_Aktiv,int64,0/680311,2


In [27]:
booking_raw.sort_values("Kamp_Erfass_Jahr", ascending=False).head(5)

,Endkunde_NR,Endkunde,EK_Abc,EK_Boni,EK_Plz,EK_Ort,EK_Land,EK_HB_Apg_Kurzz,EK_Kam_Betreut,EK_Aktiv,Agentur,AG_Hauptbetreuer,Verkaufsberater,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Endkunde_Nbranchengruppe_ID,Endkunde_Nbranchengruppe,Endkunde_Branchenkat_ID,Endkunde_Branchenkat,Endkunde_Nbranchenkat_ID,Endkunde_Nbranchenkat,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe,Auftrag_Nbranchengruppe_ID,Auftrag_Nbranchengruppe,Auftrag_Branchenkat_ID,Auftrag_Branchenkat,Auftrag_Nbranchenkat_ID,Auftrag_Nbranchenkat,Agps_NR,Segment,KV_NR,KV_Typ,Kampagnen_Status,Kampagne_Erfassungsdatum,Kampagne_Beginn,Auftragsart,Res_Dat,Annullation_Datum,Aush_Von,Dauer,Vertrag,Brutto,Netto,Agglo,PF,Kamp_Beginn_Jahr,Kamp_Beginn_KW,Kamp_Beginn_KW_2,Kamp_Beginn_KW_4,Kamp_Erfass_Jahr,Kamp_Erfass_KW,Kamp_Erfass_KW_2,Kamp_Erfass_KW_4
795283,105729,Schweizerische Mobiliar,KA,gut,3001,Bern,SCHWEIZ,ROE,1,1,mediaschneider ag,PKL,STL,475,WG - Versicherungen,NaN,NaN,12,WB - Finanzwirtschaft / Versicherung,NaN,NaN,475,WG - Versicherungen,NaN,NaN,12,WB - Finanzwirtschaft / Versicherung,NaN,NaN,2989063,APG|SGA,1002634,KPGL,4,2019-06-27,2019-07-15,Kommerziell,2019-06-27,NaT,2019-07-15,28,Nein,545,545,"90306,90329,90404,90412,90768,91103,91362,9137...","BigPoster,BrandingWall,City eBoard,City ePanel...",2019,29,29,29,2019,26,25,25
1381933,495076,ALDI SUISSE AG,A,gut,9536,Schwarzenbach SG,SCHWEIZ,ENN,1,1,Omnicom Media Group Schweiz AG,BAM,BAM,901,WG - Handel / Grossverteiler,901,WG - Handel / Grossverteiler,14,WB - Handel,14,WB - Handel,901,WG - Handel / Grossverteiler,NaN,NaN,14,WB - Handel,NaN,NaN,2915152,APG|SGA,990242,KPG,4,2019-03-11,2019-04-29,Kommerziell,2019-03-11,NaT,2019-04-29,14,Nein,9250,3183,"90306,90329,90404,90412,90768,91103,91301,9136...","City eBoard,City ePanel,Escalator ePanel,F12,F...",2019,18,17,17,2019,11,11,9
185626,491019,Preziosi Point Sagl,E,gut,6900,Lugano,SCHWEIZ,SOL,0,1,NaN,NaN,SOL,555,WG - Persönlicher Bedarf,NaN,NaN,04,WB - Bekleidung / persönlicher Bedarf,NaN,NaN,555,WG - Persönlicher Bedarf,NaN,NaN,04,WB - Bekleidung / persönlicher Bedarf,NaN,NaN,3046467,Traffic,1012412,KDVTLG,1,2019-10-01,2020-01-01,Traffic Auftrag,2019-10-01,NaT,2020-01-01,366,Ja,500,500,A5192,"F12,F200,Heckfolie,Hängekarton,Seitenbanner,Te...",2020,1,1,1,2019,40,39,37
185625,486549,Reformierte Kirchgemeinde Steffisburg,E,gut,3612,Steffisburg,SCHWEIZ,CBA,0,1,NaN,NaN,JAN,950,WG - Behörden,NaN,NaN,19,WB - Behörden / Politik,NaN,NaN,950,WG - Behörden,NaN,NaN,19,WB - Behörden / Politik,NaN,NaN,3044388,APG|SGA,1011830,KPG,1,2019-09-30,2019-11-04,Kommerziell,2019-09-30,NaT,2019-11-04,7,Nein,97,97,A0942,"F4M,F4S",2019,45,45,45,2019,40,39,37
185624,486549,Reformierte Kirchgemeinde Steffisburg,E,gut,3612,Steffisburg,SCHWEIZ,CBA,0,1,NaN,NaN,JAN,950,WG - Behörden,NaN,NaN,19,WB - Behörden / Politik,NaN,NaN,950,WG - Behörden,NaN,NaN,19,WB - Behörden / Politik,NaN,NaN,3044351,APG|SGA,1011830,KPG,1,2019-09-30,2019-11-04,Kommerziell,2019-09-30,NaT,2019-11-04,14,Nein,715,715,A0942,"F4M,F4S",2019,45,45,45,2019,40,39,37


In [28]:
booking_raw.groupby(["Vertrag","Kampagnen_Status"]).count()

Endkunde_NR  Endkunde    EK_Abc   EK_Boni    EK_Plz  \
Vertrag Kampagnen_Status                                                        
Ja      1                     12349.0   12349.0   12349.0   12349.0   12349.0   
        2                       475.0     475.0     475.0     475.0     475.0   
        3                         NaN       NaN       NaN       NaN       NaN   
        4                     85930.0   85930.0   85233.0   85930.0   85923.0   
        5                        12.0      12.0      12.0      12.0      12.0   
Nein    1                     18443.0   18443.0   18443.0   18443.0   18443.0   
        2                      7966.0    7966.0    7966.0    7966.0    7966.0   
        3                         8.0       8.0       8.0       8.0       8.0   
        4                    554958.0  554958.0  554500.0  554882.0  554800.0   
        5                       170.0     170.0     170.0     170.0     170.0   

                            EK_Ort   EK_Land  EK_HB_Apg_Kurzz  EK_Kam_Betreut  \
Vertrag Kampagnen_Status                                                        
Ja      1                  12349.0   12349.0          12278.0         12349.0   
        2                    475.0     475.0            475.0           475.0   
        3                      NaN       NaN              NaN             NaN   
        4                  85923.0   85923.0          81258.0         85930.0   
        5                     12.0      12.0             12.0            12.0   
Nein    1                  18443.0   18443.0          18428.0         18443.0   
        2                   7966.0    7966.0           7960.0          7966.0   
        3                      8.0       8.0              8.0             8.0   
        4                 554800.0  554800.0         531039.0        554958.0   
        5                    170.0     170.0            170.0           170.0   

                          EK_Aktiv   Agentur  AG_Hauptbetreuer  \
Vertrag Kampagnen_Status                                         
Ja      1                  12349.0    3987.0            3892.0   
        2                    475.0     420.0             413.0   
        3                      NaN       NaN               NaN   
        4                  85930.0   22370.0           21471.0   
        5                     12.0       0.0               0.0   
Nein    1                  18443.0   10045.0           10031.0   
        2                   7966.0    4475.0            4474.0   
        3                      8.0       2.0               2.0   
        4                 554958.0  313704.0          313131.0   
        5                    170.0     119.0             119.0   

                          Verkaufsberater  Endkunde_Branchengruppe_ID  \
Vertrag Kampagnen_Status                                                
Ja      1                         12349.0                     12327.0   
        2                           475.0                       475.0   
        3                             NaN                         NaN   
        4                         85930.0                     85018.0   
        5                            12.0                        12.0   
Nein    1                         18443.0                     18300.0   
        2                          7966.0                      7753.0   
        3                             8.0                         8.0   
        4                        554958.0                    550529.0   
        5                           170.0                       168.0   

                          Endkunde_Branchengruppe  \
Vertrag Kampagnen_Status                            
Ja      1                                 12327.0   
        2                                   475.0   
        3                                     NaN   
        4                                 85018.0   
        5                                    12.0   
Nein    1                              

In [7]:
def booking_nettos_vbs(booking_raw):
    today = pd.Timestamp.now()
    booking_raw = booking_raw.astype({"Kamp_Erfass_Jahr": "int64"})

    ##
    row_select = (#booking_raw.loc[:,"Vertrag"] == "Ja") &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] <= today.isocalendar()[0]) &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] >= (today.isocalendar()[0])-4) 
                 )

    booking_nettos = pd.pivot_table(
        booking_raw.loc[row_select,:],
        values  = ["Netto"],
        aggfunc = {"Netto": np.nansum},
        columns = ["Kamp_Erfass_Jahr"],
        index   = ["Endkunde_NR"],
        fill_value= 0
        )

    booking_nettos_flattened = pd.DataFrame(booking_nettos.to_records(index=False))
    booking_nettos_flattened.loc[:,"Endkunde_NR"] = pd.Series(booking_nettos.index)
    col_names = ["Net_"+str(x) for x in range(today.isocalendar()[0]-4, today.isocalendar()[0]+1)] + ["Endkunde_NR"]
    booking_nettos_flattened.columns = col_names

    ##
    row_select = (#booking_raw.loc[:,"Vertrag"] == "Ja") &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] <= today.isocalendar()[0]) &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] >= (today.isocalendar()[0])-2) 
                 )

    booking_letzte_vbs = (booking_raw
        .loc[row_select,:]
        .groupby("Endkunde_NR", as_index=False)
        .agg({"Verkaufsberater": collect})
        .rename({"Verkaufsberater": "Letzte_VBs"})
        )

    ## merge
    ek_booking = pd.merge(
        booking_nettos_flattened,
        booking_letzte_vbs,
        on="Endkunde_NR",
        how="outer")
    return ek_booking